get NPP from MYD MOD17AHGF

year: 2001 - 2020

Point: Japanese 4rd time Mesh 0.5 km centroid

## NNP Extraction
Net Primary Productivity

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from osgeo import gdal
import os
import glob
import rasterio

In [6]:
year = 2015
aimFolder = "F:/17_Article/01_Data/02_NetPrimaryProductivity"
os.mkdir(aimFolder + "\\temp")

In [9]:
def mergeAndResampleNppRaster(year, aimFolder):
    #"""
    #Merge MOD and MYD,
    #Resample the raster with 0.004 degrees
    #"""
    rasterFile = glob.glob("F:/17_Article/01_Data/02_NetPrimaryProductivity/Net_PP_Yearly_500m_v6/Npp/*"+str(year) +"_001.tif")

    if year > 2001:
        tifflayer_0 = gdal.Open(rasterFile[0], gdal.GA_ReadOnly)
        tifflayer_1 = gdal.Open(rasterFile[1], gdal.GA_ReadOnly)

        nband = 1
        geotransform = tifflayer_0.GetGeoTransform()
        spatialreference = tifflayer_0.GetProjection()

        tifflayer_0_array = tifflayer_0.ReadAsArray()
        tifflayer_0_array = np.array(tifflayer_0_array)
        tifflayer_0_array = tifflayer_0_array.astype(float)
        tifflayer_0_array[tifflayer_0_array == 65535] = np.nan
        tifflayer_0_array[tifflayer_0_array > 32760] = 0

        tifflayer_1_array = tifflayer_1.ReadAsArray()
        tifflayer_1_array = np.array(tifflayer_1_array)
        tifflayer_1_array = tifflayer_1_array.astype(float)
        tifflayer_1_array[tifflayer_1_array == 65535] = np.nan
        tifflayer_1_array[tifflayer_1_array > 32760] = 0

        tifflayer_double_array = np.array([tifflayer_0_array, tifflayer_1_array])
        tifflayer_mean_array = np.nanmean(tifflayer_double_array, axis = 0)
    else:
        tifflayer_0 = gdal.Open(rasterFile[0], gdal.GA_ReadOnly)

        nband = 1
        geotransform = tifflayer_0.GetGeoTransform()
        spatialreference = tifflayer_0.GetProjection()

        tifflayer_0_array = tifflayer_0.ReadAsArray()
        tifflayer_0_array = np.array(tifflayer_0_array)
        tifflayer_0_array = tifflayer_0_array.astype(float)
        tifflayer_0_array[tifflayer_0_array == 65535] = np.nan
        tifflayer_0_array[tifflayer_0_array > 32760] = 0
        tifflayer_mean_array = tifflayer_0_array 


    ncol = tifflayer_mean_array.shape[1]
    nrow = tifflayer_mean_array.shape[0]

    temp_file = aimFolder + "/temp/temp_file_" + str(year) + ".tif"

    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(temp_file, ncol, nrow, nband, gdal.GDT_Float32)
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(tifflayer_mean_array)
    dst_dataset = None

    original_temp_raster = gdal.Open(temp_file, gdal.GA_ReadOnly)
    raster_rprj = gdal.Warp(aimFolder + "/temp/temp_file_" + str(year) + "0008.tif", original_temp_raster, dstSRS = "EPSG:4326", 
                            xRes = 0.004, yRes = 0.004, resampleAlg = "average")
    original_temp_raster = None
    raster_rprj = None


In [7]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

Parallel(n_jobs=10)(delayed(mergeAndResampleNppRaster)(int(year), aimFolder) for year in np.linspace(2002, 2020, 20))

IndexError: list index out of range

In [8]:
np.linspace(2001, 2020, 20)

array([2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008., 2009.,
       2010., 2011., 2012., 2013., 2014., 2015., 2016., 2017., 2018.,
       2019., 2020.])

In [2]:
coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")

In [ ]:

for year in np.linspace(2001, 2020, 20):
    rasterFile = rasterio.open(aimFolder + "/temp/temp_file_" + str(year) + "0004.tif")
    rasterArray = rasterFile.read(1)

    valueArray = []
    for point in coords_extration['geometry']:
        x = point.xy[0][0]
        y = point.xy[1][0]
        row, col = rasterFile.index(x, y)
        valueArray.append(rasterArray[row, col])
        
    valueArray = np.array(valueArray)
    coords_extration['NPP_'+str(year)] = valueArray

    rasterFile = None

In [ ]:
#os.remove(temp_file)
#os.remove(aimFolder + "/temp/temp_file_0008.tif")

#os.rmdir(aimFolder + "\\temp")

## NTL extraction
Night Time Light

In [3]:
import os
import zipfile
import glob
import rasterio
from osgeo import gdal
import geopandas as gpd
import numpy as np

In [ ]:
aimFolder = "F:\\17_Article\\01_Data\\03_NTL_VIIRS"
os.mkdir(aimFolder + "\\temp")

year = 2001

while year < 2021:
    filename = aimFolder + "\\" + str(year) + ".zip"
    with zipfile.ZipFile(filename, "r") as zip_ref:
        zip_ref.extractall(aimFolder + "\\temp")

    rasterFile = rasterio.open(raster_out)
    aimPoint = coords_extration
    rasterArray = rasterFile.read(1)



In [4]:
aimFolder = "F:\\17_Article\\01_Data\\03_NTL_VIIRS"
os.mkdir(aimFolder + "\\temp")

In [6]:
year = 2001
filename = aimFolder + "\\" + str(year) + ".zip"
with zipfile.ZipFile(filename, "r") as zip_ref:
    zip_ref.extractall(aimFolder + "\\temp")

In [7]:
aimRaster = glob.glob(aimFolder + "\\temp\\*" + str(year) + ".tif")
aimRaster = aimRaster[0]

In [9]:
rasterFile = rasterio.open(aimRaster)
rasterArray = rasterFile.read(1)

In [10]:
valueArray = []
for point in coords_extration['geometry']:
    x = point.xy[0][0]
    y = point.xy[1][0]
    row, col = rasterFile.index(x, y)
    valueArray.append(rasterArray[row, col])
    
valueArray = np.array(valueArray)
coords_extration['NTL_'+str(year)] = valueArray